<a href="https://colab.research.google.com/github/tomrdchen/3rd-ML100Days/blob/master/%E2%80%9CSentiment_Analysis%E2%80%9D%E7%9A%84%E5%89%AF%E6%9C%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!wget "https://www.dropbox.com/s/w03mdyw4kqimwgt/data.zip?dl=0"
!unzip data.zip\?dl\=0

In [0]:
import csv
import torch
import itertools
from torch.utils.data import Dataset, DataLoader

class Voc:
    def __init__(self, size=4, lines=[]):
        assert size >= 4
        self.index2word = {0: "SOS", 1: "EOS", 2:"PAD", 3:"UNK"}
        word2count = {}
        for l in lines:
            for word in l.split(' '):
                if word not in word2count:
                    word2count[word] = 1
                else:
                    word2count[word] += 1
        word2count = list(word2count.items())
        word2count.sort(key=lambda x: x[1], reverse=True)
        size = min(size, len(word2count))
        if len(lines):
            print("{} words trimmed to {} words".format(len(word2count), size))
        for i in range(size-4):
            self.index2word[i+4] = word2count[i][0]
        self.word2index = {v: k for k, v in self.index2word.items()}

    def getIndex(self, word):
        if word in self.word2index:
            return self.word2index[word]
        else:
            return self.word2index["UNK"]

    def save2file(self, path):
        with open(path, 'w') as f:
            yaml.dump(self.index2word, f, default_flow_style=False, allow_unicode=True)

    def load_file(self, path):
        with open(path, 'r') as f:
            self.index2word = yaml.load(f)
            self.word2index = {v: k for k, v in self.index2word.items()}
            self.size = len(self.index2word)

class Twit_dataset(Dataset):
    def __init__(self, csv_file, voc):
        with open(csv_file) as f:
            reader = csv.reader(f)
            self.data = list(reader)
        self.data = [(int(d[0]), [voc.getIndex(w) for w in d[1]]) for d in self.data]

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

def collate_fn(batch):
    def pad(seqs, fillvalue=2):
        tmp = list(itertools.zip_longest(*seqs, fillvalue=fillvalue))
        return torch.LongTensor(tmp)
    ys, xs = list(zip(*batch))
    xs = pad(xs)
    return torch.LongTensor(xs), torch.FloatTensor(ys)

seqs = []
size = 20000
with open('data/train.csv') as f:
    reader = csv.reader(f)
    for r in reader:
        seqs.append(r[1])
voc = Voc(size, seqs)
train_dataset = Twit_dataset('data/train.csv', voc)
valid_dataset = Twit_dataset('data/test.csv', voc)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=1, collate_fn=collate_fn)
valid_loader = DataLoader(valid_dataset, batch_size=64, shuffle=False, num_workers=1, collate_fn=collate_fn)

In [0]:
import torch.nn as nn

class My_model(nn.Module):
    def __init__(self, n_layers, voc_size, dropout=0.2):
        super().__init__()
        self.embedding = nn.Embedding(voc_size, 256)
        self.rnn = nn.GRU(256, 256, n_layers, dropout=dropout)
        self.fc = nn.Linear(256, 1)

    def forward(self, x):
        emb = self.embedding(x)
        output, hidden = self.rnn(emb)
        output = self.fc(output[-1])
        return torch.sigmoid(output)

In [0]:
device = 'cuda'
n_epoch = 5
model = My_model(2, 20000).to(device)
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
print_loss = 0.0

In [0]:
for e in range(n_epoch):
    print('============= Epoch: {} ============='.format(e+1))
    for idx, (x, y) in enumerate(train_loader):
        x = x.to(device)
        y = y.to(device)
        o = model(x)
        loss = criterion(o.squeeze(), y)
        print_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if (idx+1) % 100 == 0:
            print('loss:', print_loss/100)
            print_loss = 0.0
